<h1># Open again in assistant editor to compare</h1>

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import IPython.display
import numpy as np
from cued_sf2_lab.familiarisation import load_mat_img, plot_image
from cued_sf2_lab.jpeg import jpegdec, jpegenc
from scipy import optimize

In [ ]:
lighthouse, _ = load_mat_img(img='lighthouse.mat', img_info='X')
bridge, _ = load_mat_img(img='bridge.mat', img_info='X')
flamingo, _ = load_mat_img(img='flamingo.mat', img_info='X')

lighthouse = lighthouse - 128.0
bridge = bridge - 128.0
flamingo = flamingo - 128.0

In [ ]:
image = lighthouse # flamingo breaks variation of LBT and DWT
size_lim = 40906 - 1440 - 10

<h1>Raw</h1>

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))

plot_image(image, ax=ax)
ax.set(yticks=[], xticks=[])

# fig.savefig("figs/l_raw.pdf")

print(f'Size = {256*256*8}, Err = {np.std(0):.4}')

<h1>JPEG (slow)</h1>

In [ ]:
%%script echo skipping

def error(qstep: int) -> int:

    Z, h = jpegenc(image, qstep, opthuff=True, dcbits=16, log=False)
    size = Z[:, 1].sum()
    return np.sum((size - size_lim)**2)

opt_step = optimize.minimize_scalar(error, method="bounded", bounds=(4, 128)).x
vlc, hufftab = jpegenc(image, opt_step, opthuff=True, dcbits=10, log=False)

Z = jpegdec(vlc, opt_step, hufftab=hufftab, dcbits=10, log=False)

fig, ax = plt.subplots(figsize=(20, 20))
plot_image(Z, ax=ax)
ax.set(yticks=[], xticks=[])

# fig.savefig("figs/l_jpeg_opthuff_dc16bits.pdf")


print(f'Size = {vlc[:, 1].sum()}, Step = {opt_step:.4} (check bounds), Err = {np.std(Z - image):.4}')

<h1>DWT</h1>

In [ ]:

from compression import DWTCompression, LBTCompression

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
DWT = DWTCompression(5)
M = 16
Y1 = DWT.compress(image)

(vlc, hufftab), qs = DWT.opt_encode(Y1, size_lim=size_lim, M=M, root2=False)

Y = DWT.decode(vlc, qstep=qs, hufftab=hufftab, M=M, root2=False)

Z = DWT.decompress(Y)

plot_image(Z, ax=ax)
print(f'Size = {vlc[:, 1].sum()}, Step = {qs:.4} (check bounds), Err = {np.std(Z - image):.4}')

ax.set(yticks=[], xticks=[])

# fig.savefig("figs/l_dwt_opthuff.pdf")

<h1>DWT | Root 2</h1>

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
image = lighthouse
DWT = DWTCompression(4) # 5 layers breaks saying code size is too big

Y = DWT.compress(image)

(vlc, hufftab), qs = DWT.opt_encode(Y, size_lim=size_lim, M=16, rise_ratio=0.5)

Y = DWT.decode(vlc, qstep=qs, hufftab=hufftab, M=16, rise_ratio=0.5)

Z = DWT.decompress(Y)

plot_image(Z, ax=ax)
print(f'Size = {vlc[:, 1].sum()}, Step = {qs:.4} (check bounds), Err = {np.std(Z - image):.4}')

ax.set(yticks=[], xticks=[])

# fig.savefig("figs/l_dwt_opthuff_root2.pdf")


<h1> DWT | Root 2 | Rise Ratio </h1>

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))
image = lighthouse
DWT = DWTCompression(4)
RR = 0.68
M = 16

Y = DWT.compress(image)

(vlc, hufftab), qs = DWT.opt_encode(Y, size_lim=size_lim, M=M, rise_ratio=RR)

Y = DWT.decode(vlc, qstep=qs, hufftab=hufftab, M=M, rise_ratio=RR)

Z = DWT.decompress(Y)

plot_image(Z, ax=ax)
print(f'Size = {vlc[:, 1].sum()}, Step = {qs:.4} (check bounds), Err = {np.std(Z - image):.4}')

ax.set(yticks=[], xticks=[])

# fig.savefig("figs/l_dwt_opthuff_root2.pdf")

<h1>LBT</h1>

In [ ]:
M = 16

scheme = LBTCompression(s=1.3, N=16)

fig, ax = plt.subplots(figsize=(20, 20))

Y = scheme.compress(image)
(vlc, hufftab), qs = scheme.opt_encode(Y, size_lim=size_lim, M=M)
header = (hufftab, qs)

Y = scheme.decode(vlc, qstep=header[1], M=M, hufftab=header[0])
Z = scheme.decompress(Y)

plot_image(Z, ax=ax)
print(f'Size = {vlc[:, 1].sum()}, Step = {qs:.4} (check bounds), Err = {np.std(Z - image):.4}')

ax.set(yticks=[], xticks=[])

# fig.savefig("figs/l_lbt_opthuff.pdf")

<h1>LBT with Quantisation Gradient</h1>

In [ ]:
# Quantisation gradient top left to bottom right, apparently just lightly high passes but introduces massive RMS error
# unreliable. M = 8 for flamingo, = 16 for lighthouse

M = 16

image = lighthouse

scheme = LBTCompression(s=1.3, N=M)

fig, ax = plt.subplots(figsize=(20, 20))

Y = scheme.compress(image)
(vlc, hufftab), qs = scheme.opt_encode(Y, size_lim=size_lim, M=M, quant_grad=True)
header = (hufftab, qs)

Y = scheme.decode(vlc, qstep=header[1], M=M, hufftab=header[0], quant_grad=True)
Z = scheme.decompress(Y)
plot_image(Z, ax=ax)
print(f'Size = {vlc[:, 1].sum()}, Step = {qs:.4} (check bounds), Err = {np.std(Z - image):.4}')

ax.set(yticks=[], xticks=[])

# fig.savefig("figs/l_lbt_opthuff_qgrad.pdf")